In [11]:
from random import gauss
import os 
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
sys.path.insert(2,'..')
import functions
import xgboost as xgb
from sklearn import preprocessing
import annualized_rv as arv
from sklearn.model_selection import TimeSeriesSplit

In [4]:
aaplHistIV = pd.read_pickle('historicImpliedVolData/aapl_mean_iv_2017_2022.pkl')
googHistIV = pd.read_pickle('historicImpliedVolData/goog_mean_iv_2017_2022.pkl')
msftHistIV = pd.read_pickle('historicImpliedVolData/msft_mean_iv_2017_2022.pkl')
ndxHistIV = pd.read_pickle('historicImpliedVolData/ndx_mean_iv_2017_2022.pkl')
spyHistIV = pd.read_pickle('historicImpliedVolData/spc_mean_iv_2017_2022.pkl')


In [5]:
spyHistIV.rename(columns = {'date':'Date'},inplace = True) #renaming date column to Date for consistency
spyHistIV.set_index('Date',inplace = True,drop = True) #setting index to date

In [6]:
#grab spy  from yfinance

spyHistory = yf.download('^GSPC', start='2016-01-01', end='2023-12-31')
#calculate realised vol
window =21 #realisedVol window size
spyHistory['Daily Return'] = spyHistory['Adj Close'].pct_change()
spyHistory['21dRealisedVol'] = spyHistory['Daily Return'].rolling(window=window).std() * np.sqrt(252)


[*********************100%%**********************]  1 of 1 completed


In [7]:
historicIVSeries = spyHistIV['average_iv']
historicVolumeSeries = spyHistory['Volume'].rolling(21).mean()['2017':'2021']
dailyReturnSeries= spyHistory['Daily Return']['2017':'2021']
dailyRealisedVolSeries = spyHistory['21dRealisedVol']['2017':'2021']
df_combined = pd.concat([historicIVSeries,dailyReturnSeries,dailyRealisedVolSeries,historicVolumeSeries], axis=1)

# Scale the data
scaler = preprocessing.StandardScaler().fit(df_combined)
scaled_data = scaler.transform(df_combined)

In [8]:
data = scaled_data
seq_length = 60
n_features = data.shape[1]


In [9]:
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length - 3):
        x = data[i:(i + seq_length)]
        y = data[(i + seq_length), 0]  # Next 3-day IV
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [12]:
X, y = create_sequences(data, seq_length)

tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]



1196